# 🛸 Drohnen-Detektion in Drohnen‑Videos mit **YOLOv8** (Google Colab)
**Super‑kommentierte Schritt‑für‑Schritt‑Anleitung** für 5.‑Semester‑Studis: Installation → Datenaufbereitung (Frames + 640×640‑Kacheln) → Training → Evaluation → Inference (inkl. **tiled inference** für 4K‑Videos) → Export zurück in Google Drive.

> **Tipp:** In Colab unter *Runtime → Change runtime type → Hardware accelerator → GPU* eine GPU wählen (T4/A100).  
> Diese Notebook‑Version generiert keine synthetischen Daten; für Annotationen (Labels) nutzt bitte Tools wie **CVAT**, **Label Studio** oder **Roboflow Annotate** und exportiert im **YOLO‑TXT‑Format**.

## 🚀 Quick Start (Überblick)
1. **Installieren** (`ultralytics`, `opencv-python`, `ffmpeg`)  
2. **Google Drive mounten** und Pfade setzen  
3. **Videos aus Drive einbinden** → `raw_videos/`  
4. **Frames extrahieren** (z. B. 2 FPS)  
5. **In 640×640 kacheln** (20 % Überlappung)  
6. **Kacheln labeln** (extern), Labels ins Projekt kopieren  
7. **Datensatz (train/val/test) bauen** + `data.yaml`  
8. **YOLOv8 trainieren** und **validieren**  
9. **Inference** (einfach & tiled für 4K)  
10. **Ergebnisse nach Drive sichern**

## 0️⃣ Umgebung prüfen (GPU)

pip install torch

In [2]:
# !nvidia-smi || true

import torch, platform, sys
print("Python:", sys.version.split()[0])
print("PyTorch:", torch.__version__)
print("CUDA verfügbar:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Gerät:", torch.cuda.get_device_name(0))
else:
    print("⚠️ Keine GPU aktiv. CPU funktioniert auch, ist aber langsamer. In Colab ggf. GPU aktivieren (Runtime → Change runtime type).")

Python: 3.13.2
PyTorch: 2.9.1+cpu
CUDA verfügbar: False
⚠️ Keine GPU aktiv. CPU funktioniert auch, ist aber langsamer. In Colab ggf. GPU aktivieren (Runtime → Change runtime type).


## 1️⃣ Installation (YOLOv8 + Tools)

pip install ultralytics opencv-python tqdm numpy matplotlib

In [4]:
!pip -q install ultralytics opencv-python tqdm numpy matplotlib
!apt-get -qq update
!apt-get -qq install -y ffmpeg

import shutil
print("Pfad zu 'yolo':", shutil.which("yolo"))

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Pfad zu 'yolo': C:\Users\Ignaz\.conda\envs\Schmetterling\Scripts\yolo.EXE


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [5]:
import os, glob, math, shutil, subprocess, random, json, textwrap, time
from pathlib import Path
from typing import List, Tuple
import numpy as np
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt

import ultralytics
from ultralytics import YOLO

print("ultralytics:", ultralytics.__version__)
print("opencv:", cv2.__version__)

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\Ignaz\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
ultralytics: 8.3.229
opencv: 4.12.0


## 2️⃣ Google Drive mounten & Projektpfade setzen
- Legt eure Videos in Drive z. B. unter: `MyDrive/datasets/drone_videos`  
- **Anpassen:** `GDRIVE_VIDEOS_DIR` unten ggf. auf euren Pfad ändern.

google drive eig. nicht mehr nötig

In [20]:
GDRIVE_VIDEOS_DIR = "/datasets/drone_videos/"
DRIVE_EXPORT_DIR  = "/drone_yolo8_runs/"

PROJ_ROOT   = "../Schmetterlingsverfolgung"
RAW_VIDEOS  = f"{PROJ_ROOT}/raw_videos"
FRAMES_RAW  = f"{PROJ_ROOT}/frames_raw"
FRAMES_TILES= f"{PROJ_ROOT}/frames_tiles"
ANNOTATIONS = f"{PROJ_ROOT}/annotations"
DATASET     = f"{PROJ_ROOT}/datasets"
TOOLS       = f"{PROJ_ROOT}/tools"

for p in [
    PROJ_ROOT, RAW_VIDEOS, FRAMES_RAW, FRAMES_TILES, ANNOTATIONS, DATASET,
    f"{DATASET}/images/train", f"{DATASET}/images/val", f"{DATASET}/images/test",
    f"{DATASET}/labels/train", f"{DATASET}/labels/val", f"{DATASET}/labels/test",
    DRIVE_EXPORT_DIR, TOOLS
]:
    os.makedirs(p, exist_ok=True)

print("Projektordner:", PROJ_ROOT)
print("Drive Videos:", GDRIVE_VIDEOS_DIR)
print("Drive Export:", DRIVE_EXPORT_DIR)

Projektordner: ../Schmetterlingsverfolgung
Drive Videos: /datasets/drone_videos/
Drive Export: /drone_yolo8_runs/


## 3️⃣ Videos aus Google Drive einbinden
- Standard: **Symlink** (spart Speicher). Fallback: **Copy**.

In [21]:
def collect_videos(src_dir: str, dst_dir: str) -> list:
    exts = ("*.MP4","*.mov","*.mkv","*.avi","*.m4v")
    videos = []
    for e in exts:
        videos.extend(glob.glob(os.path.join(src_dir, e)))
    videos = sorted(videos)
    print(f"Gefundene Videos in Drive: {len(videos)}")
    for v in videos[:5]:
        print("  •", os.path.basename(v))
    for v in videos:
        name = os.path.basename(v)
        dst = os.path.join(dst_dir, name)
        if os.path.exists(dst):
            continue
        try:
            os.symlink(v, dst)
            mode = "link"
        except Exception:
            shutil.copy2(v, dst)
            mode = "copy"
        print(f"{mode:4s} → {name}")
    return sorted(glob.glob(os.path.join(dst_dir, "*")))

videos_local = collect_videos(GDRIVE_VIDEOS_DIR, RAW_VIDEOS)
print("Videos im Projekt:", len(videos_local))

Gefundene Videos in Drive: 0
Videos im Projekt: 2


## 4️⃣ Frames aus Videos extrahieren (sparsam samplen)
- Empfehlung: **2 FPS** (anpassbar über `FPS`).

conda install -c conda-forge ffmpeg

In [26]:
FPS = 2

def extract_frames(video_path: str, out_dir: str, fps: int = 2):
    os.makedirs(out_dir, exist_ok=True)

    # Pfade bereinigen
    video_path = os.path.normpath(video_path)
    out_dir = os.path.normpath(out_dir)
    out_pattern = os.path.normpath(os.path.join(out_dir, "frame_%06d.jpg"))


    out_pattern = os.path.join(out_dir, "frame_%06d.jpg")
    cmd = ["ffmpeg", "-y", "-i", video_path, "-vf", f"fps={fps}", out_pattern]
    print("FFmpeg:", " ".join(cmd))
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if result.returncode != 0:
        print("⚠️ FFmpeg Fehler:", result.stderr[:2000])

for vp in videos_local:
    base = os.path.splitext(os.path.basename(vp))[0]
    out = os.path.join(FRAMES_RAW, base)
    if len(glob.glob(os.path.join(out, "*.jpg"))) > 0:
        print("Skip (bereits extrahiert):", base)
        continue
    extract_frames(vp, out, FPS)

folders = sorted([d for d in glob.glob(os.path.join(FRAMES_RAW, "*")) if os.path.isdir(d)])
total_frames = sum(len(glob.glob(os.path.join(d, "*.jpg"))) for d in folders)
print("Ordner mit Frames:", len(folders))
print("Gesamtzahl Frames:", total_frames)

FFmpeg: ffmpeg -y -i ..\Schmetterlingsverfolgung\raw_videos\DJI_0519.MP4 -vf fps=2 ..\Schmetterlingsverfolgung\frames_raw\DJI_0519\frame_%06d.jpg
FFmpeg: ffmpeg -y -i ..\Schmetterlingsverfolgung\raw_videos\DJI_0521.MP4 -vf fps=2 ..\Schmetterlingsverfolgung\frames_raw\DJI_0521\frame_%06d.jpg
Ordner mit Frames: 2
Gesamtzahl Frames: 27


## 5️⃣ 4K‑Frames in **640×640‑Kacheln** schneiden (+ Überlappung)
**Empfehlung:** `TILE_SIZE = 640`, `OVERLAP = 0.2` (20 %).

In [27]:
TILE_SIZE = 640
OVERLAP   = 0.20
JPEG_QUALITY = 95

from typing import List, Tuple

def make_grid_coords(w: int, h: int, tile: int, overlap: float) -> Tuple[List[int], List[int]]:
    stride = int(tile * (1 - overlap))
    xs = list(range(0, max(w - tile, 0) + 1, stride))
    ys = list(range(0, max(h - tile, 0) + 1, stride))
    if xs[-1] != w - tile: xs.append(max(w - tile, 0))
    if ys[-1] != h - tile: ys.append(max(h - tile, 0))
    return xs, ys

def tile_image(img_path: str, dst_dir: str, tile: int = 640, overlap: float = 0.2) -> int:
    img = cv2.imread(img_path)
    if img is None:
        return 0
    h, w = img.shape[:2]
    xs, ys = make_grid_coords(w, h, tile, overlap)
    base = os.path.splitext(os.path.basename(img_path))[0]
    sub  = os.path.basename(os.path.dirname(img_path))
    out_dir = os.path.join(dst_dir, sub)
    os.makedirs(out_dir, exist_ok=True)
    count = 0
    for y in ys:
        for x in xs:
            tile_img = img[y:y+tile, x:x+tile]
            out_name = f"{base}_x{x}_y{y}.jpg"
            out_path = os.path.join(out_dir, out_name)
            cv2.imwrite(out_path, tile_img, [int(cv2.IMWRITE_JPEG_QUALITY), JPEG_QUALITY])
            count += 1
    return count

total_tiles = 0
frame_folders = sorted([d for d in glob.glob(os.path.join(FRAMES_RAW, "*")) if os.path.isdir(d)])
for folder in frame_folders:
    frames = sorted(glob.glob(os.path.join(folder, "*.jpg")))
    print(f"Kacheln: {os.path.basename(folder)}  (Frames: {len(frames)})")
    for p in tqdm(frames, ncols=100):
        total_tiles += tile_image(p, FRAMES_TILES, TILE_SIZE, OVERLAP)

print("Gesamtzahl Kacheln:", total_tiles)

Kacheln: DJI_0519  (Frames: 12)


100%|███████████████████████████████████████████████████████████████| 12/12 [00:01<00:00, 11.18it/s]


Kacheln: DJI_0521  (Frames: 15)


100%|███████████████████████████████████████████████████████████████| 15/15 [00:01<00:00,  8.81it/s]

Gesamtzahl Kacheln: 864


## 6️⃣ **Annotieren (Labels zeichnen)** – außerhalb von Colab
1. Tool wählen: **CVAT**, **Label Studio** oder **Roboflow Annotate**.  
2. **Kacheln** aus `frames_tiles/` importieren.  
3. Klassen: für den Anfang nur **`0 drone`**.  
4. Export im **YOLO‑TXT‑Format** (eine `.txt` pro Bild).  
5. **`.txt`** in **`ANNOTATIONS`** kopieren (Basisname muss exakt zum Bild passen).

## 7️⃣ Labels überprüfen (Abdeckung & Konsistenz)
- Zählt, wie viele Bilder **mit / ohne** Label vorhanden sind.  
- Optional: Für fehlende Labels leere Dateien anlegen (negative Beispiele).

In [ ]:
ALLOW_EMPTY_LABELS = True

tile_imgs = sorted(glob.glob(f"{FRAMES_TILES}/**/*.jpg", recursive=True))
print("Anzahl Kachel-Bilder:", len(tile_imgs))

missing = 0
nonempty = 0
total_labels = 0
for img in tile_imgs:
    base = os.path.splitext(os.path.basename(img))[0]
    lbl = os.path.join(ANNOTATIONS, base + ".txt")
    if not os.path.exists(lbl):
        missing += 1
        if ALLOW_EMPTY_LABELS:
            open(lbl, "w").close()
    else:
        with open(lbl, "r") as f:
            lines = [ln.strip() for ln in f.readlines() if ln.strip()]
            if len(lines) > 0:
                nonempty += 1
                total_labels += len(lines)

print(f"Fehlende Labels (vor ggf. Erstellen leerer Dateien): {missing}")
print(f"Bilder mit >=1 Labelzeile: {nonempty}")
print(f"Summe aller Labelzeilen: {total_labels}")
if ALLOW_EMPTY_LABELS:
    now_missing = sum(1 for img in tile_imgs if not os.path.exists(os.path.join(ANNOTATIONS, os.path.splitext(os.path.basename(img))[0] + '.txt')))
    print(f"Fehlende Labels (nach Anlegen leerer Dateien): {now_missing}")

## 8️⃣ Datensatz bauen: **train/val/test** + `data.yaml`
- Standard Split: **80 % / 10 % / 10 %**. Negative Beispiele werden behalten.

In [ ]:
import random
random.seed(0)
SPLITS = {"train": 0.8, "val": 0.1, "test": 0.1}

for s in ["images/train","images/val","images/test","labels/train","labels/val","labels/test"]:
    p = os.path.join(DATASET, s)
    os.makedirs(p, exist_ok=True)

all_imgs = tile_imgs.copy()
random.shuffle(all_imgs)
n = len(all_imgs)
n_train = int(n * SPLITS["train"])
n_val   = int(n * SPLITS["val"])
train_imgs = all_imgs[:n_train]
val_imgs   = all_imgs[n_train:n_train+n_val]
test_imgs  = all_imgs[n_train+n_val:]

def copy_pair(img_list, split):
    for img in img_list:
        base = os.path.splitext(os.path.basename(img))[0]
        lbl  = os.path.join(ANNOTATIONS, base + ".txt")
        if not os.path.exists(lbl):
            if ALLOW_EMPTY_LABELS:
                open(lbl, "w").close()
            else:
                continue
        shutil.copy2(img, os.path.join(DATASET, f"images/{split}", os.path.basename(img)))
        shutil.copy2(lbl, os.path.join(DATASET, f"labels/{split}", base + ".txt"))

copy_pair(train_imgs, "train")
copy_pair(val_imgs,   "val")
copy_pair(test_imgs,  "test")

print("Split Größen:")
for split in ["train","val","test"]:
    ni = len(glob.glob(os.path.join(DATASET, f"images/{split}/*.jpg")))
    nl = len(glob.glob(os.path.join(DATASET, f"labels/{split}/*.txt")))
    print(f"  {split:5s}  imgs={ni:6d}  labels={nl:6d}")

DATA_YAML = os.path.join(PROJ_ROOT, "data.yaml")
data_yaml_text = f\"\"\"# YOLOv8 data.yaml (auto-generated)
path: {DATASET}
train: images/train
val: images/val
test: images/test

names:
  0: drone
\"\"\"
with open(DATA_YAML, "w") as f:
    f.write(data_yaml_text)

print("data.yaml gespeichert:", DATA_YAML)
print("\\n--- data.yaml ---\\n", data_yaml_text)

## 9️⃣ Visueller Sanity‑Check (ein paar Beispiele zeichnen)

In [ ]:
def load_yolo_labels(lbl_path: str, img_w: int, img_h: int):
    boxes = []
    if not os.path.exists(lbl_path):
        return boxes
    with open(lbl_path, "r") as f:
        for ln in f:
            ln = ln.strip()
            if not ln:
                continue
            parts = ln.split()
            if len(parts) != 5 and len(parts) != 6:
                continue
            cls = int(float(parts[0]))
            cx, cy, w, h = map(float, parts[1:5])
            x1 = int((cx - w/2) * img_w); y1 = int((cy - h/2) * img_h)
            x2 = int((cx + w/2) * img_w); y2 = int((cy + h/2) * img_h)
            boxes.append((cls, x1, y1, x2, y2))
    return boxes

def show_random_samples(split="train", num=4):
    imgs = glob.glob(os.path.join(DATASET, f"images/{split}/*.jpg"))
    random.shuffle(imgs); imgs = imgs[:num]
    for p in imgs:
        img = cv2.imread(p)
        if img is None: continue
        h, w = img.shape[:2]
        base = os.path.splitext(os.path.basename(p))[0]
        lblp = os.path.join(DATASET, f"labels/{split}/{base}.txt")
        boxes = load_yolo_labels(lblp, w, h)
        vis = img.copy()
        for cls, x1, y1, x2, y2 in boxes:
            cv2.rectangle(vis, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(vis, f"drone", (x1, max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        plt.figure(); plt.imshow(cv2.cvtColor(vis, cv2.COLOR_BGR2RGB)); plt.title(os.path.basename(p)); plt.axis('off')

show_random_samples("train", num=4)

## 🔟 Training mit YOLOv8
- Standard: **CLI** (`yolo task=detect ...`)  
- Fallback: **Python‑API**, falls CLI fehlt.

In [ ]:
EPOCHS = 100
IMGSZ  = 640
BATCH  = 16

def latest_run_dir():
    runs = sorted(glob.glob("/content/runs/detect/train*"), key=os.path.getmtime)
    return runs[-1] if runs else None

import shutil, torch, os
yolo_cli = shutil.which("yolo")
if yolo_cli:
    device_arg = " device=0" if torch.cuda.is_available() else ""
    cmd = f"yolo task=detect mode=train model=yolov8n.pt data={DATA_YAML} epochs={EPOCHS} imgsz={IMGSZ} batch={BATCH}{device_arg}"
    print("CLI:", cmd)
    exit_code = os.system(cmd)
    if exit_code != 0:
        print("⚠️ CLI fehlgeschlagen. Fallback: Python-API")
        yolo_cli = None
else:
    print("⚠️ 'yolo' CLI nicht gefunden. Verwende Python-API.")

if not yolo_cli:
    model = YOLO("yolov8n.pt")
    results = model.train(
        data=DATA_YAML,
        epochs=EPOCHS,
        imgsz=IMGSZ,
        batch=BATCH,
        device=0 if torch.cuda.is_available() else "cpu",
    )

run_dir = latest_run_dir()
print("Aktueller Run-Ordner:", run_dir)

## 1️⃣1️⃣ Validierung (mAP, Precision, Recall)

In [ ]:
def find_best_weights():
    runs = sorted(glob.glob("/content/runs/detect/train*"), key=os.path.getmtime)
    for r in reversed(runs):
        p = os.path.join(r, "weights", "best.pt")
        if os.path.exists(p):
            return p
    return None

best_weights = find_best_weights()
print("Best Weights:", best_weights)

if best_weights:
    if shutil.which("yolo"):
        cmd = f"yolo task=detect mode=val model='{best_weights}' data={DATA_YAML} imgsz={IMGSZ}"
        print("CLI:", cmd); os.system(cmd)
    else:
        model = YOLO(best_weights)
        metrics = model.val(data=DATA_YAML, imgsz=IMGSZ)
else:
    print("⚠️ Keine Weights gefunden.")

## 1️⃣2️⃣ Trainingskurven ansehen

In [ ]:
def plot_training_curves(run_dir: str):
    csv_path = os.path.join(run_dir, "results.csv")
    if not os.path.exists(csv_path):
        print("⚠️ results.csv nicht gefunden:", csv_path); return
    import csv, numpy as np, matplotlib.pyplot as plt
    cols = {}
    with open(csv_path, "r") as f:
        reader = csv.DictReader(f); rows = list(reader)
        if not rows: print("⚠️ results.csv leer"); return
        keys = reader.fieldnames
        for k in keys:
            cols[k] = [float(r[k]) if r[k] != "" else np.nan for r in rows]
    for key in ["train/box_loss","train/cls_loss","metrics/mAP50(B)","metrics/mAP50-95(B)"]:
        if key in cols:
            plt.figure(); plt.plot(cols[key]); plt.title(key); plt.xlabel("epoch"); plt.ylabel(key.split("/")[-1]); plt.grid(True)

runs = sorted(glob.glob("/content/runs/detect/train*"), key=os.path.getmtime)
if runs:
    run_dir = runs[-1]; print("Plot aus:", run_dir); plot_training_curves(run_dir)
else:
    print("⚠️ Kein Trainingslauf gefunden.")

## 1️⃣3️⃣ Inference (einfach): Bilder & Videos
Für 4K oder sehr kleine Objekte → **tiled inference** nutzen.

In [ ]:
CONF = 0.25
VID_STRIDE = 2

best_weights = None
# reuse the helper
def find_best_weights():
    runs = sorted(glob.glob("/content/runs/detect/train*"), key=os.path.getmtime)
    for r in reversed(runs):
        p = os.path.join(r, "weights", "best.pt")
        if os.path.exists(p):
            return p
    return None

best_weights = find_best_weights()
if not best_weights:
    print("⚠️ Kein best.pt gefunden. Bitte Training laufen lassen.")
else:
    source_imgs = os.path.join(DATASET, "images/test")
    print("Predict auf Bildern:", source_imgs)
    if shutil.which("yolo"):
        cmd = f"yolo task=detect mode=predict model='{best_weights}' source='{source_imgs}' imgsz={IMGSZ} conf={CONF} save=True"
        print("CLI:", cmd); os.system(cmd)
    else:
        model = YOLO(best_weights); model.predict(source=source_imgs, imgsz=IMGSZ, conf=CONF, save=True, verbose=False)

    test_videos = sorted(glob.glob(os.path.join(RAW_VIDEOS, "*")))
    if test_videos:
        test_video = test_videos[0]
        print("Predict auf Video:", os.path.basename(test_video))
        if shutil.which("yolo"):
            cmd = f"yolo task=detect mode=predict model='{best_weights}' source='{test_video}' imgsz={IMGSZ} conf={CONF} vid_stride={VID_STRIDE} save=True"
            print("CLI:", cmd); os.system(cmd)
        else:
            model = YOLO(best_weights); model.predict(source=test_video, imgsz=IMGSZ, conf=CONF, vid_stride=VID_STRIDE, save=True, verbose=False)
    else:
        print("⚠️ Kein Video in", RAW_VIDEOS)

## 1️⃣4️⃣ **Tiled Inference** für 4K‑Videos
Teilt jeden Frame in 640×640‑Kacheln, führt Detektion pro Kachel durch und fusioniert Ergebnisse (NMS).

In [ ]:
def tiles_for_shape(w, h, tile, overlap):
    stride = int(tile * (1 - overlap))
    xs = list(range(0, max(w - tile, 0) + 1, stride))
    ys = list(range(0, max(h - tile, 0) + 1, stride))
    if xs[-1] != w - tile: xs.append(max(w - tile, 0))
    if ys[-1] != h - tile: ys.append(max(h - tile, 0))
    return xs, ys

def nms_boxes_xyxy(boxes, scores, conf_thr=0.25, iou_thr=0.5):
    rects = []
    for (x1,y1,x2,y2) in boxes:
        rects.append([int(x1), int(y1), int(x2-x1), int(y2-y1)])
    idxs = cv2.dnn.NMSBoxes(rects, list(map(float, scores)), conf_thr, iou_thr)
    keep = set()
    if len(idxs) > 0:
        for i in idxs.flatten(): keep.add(i)
    return keep

def tiled_inference_video(model_path, src_video, dst_video,
                          tile=640, overlap=0.2, conf=0.25, iou=0.5):
    model = YOLO(model_path)
    cap = cv2.VideoCapture(src_video); assert cap.isOpened(), f"Video nicht geöffnet: {src_video}"
    w  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)); h  = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps= cap.get(cv2.CAP_PROP_FPS) or 30.0
    fourcc = cv2.VideoWriter_fourcc(*"mp4v"); out = cv2.VideoWriter(dst_video, fourcc, fps, (w,h))
    xs, ys = tiles_for_shape(w, h, tile, overlap)
    from tqdm import tqdm; pbar = tqdm(total=int(cap.get(cv2.CAP_PROP_FRAME_COUNT) or 0), ncols=100, desc="Tiled Inference")
    while True:
        ok, frame = cap.read()
        if not ok: break
        det_boxes, det_scores, det_classes = [], [], []
        for y in ys:
            for x in xs:
                tile_img = frame[y:y+tile, x:x+tile]
                res = model.predict(source=tile_img, imgsz=tile, conf=conf, iou=iou, verbose=False)[0]
                if res.boxes is None: continue
                for b in res.boxes:
                    xyxy = b.xyxy[0].cpu().numpy(); x1, y1, x2, y2 = xyxy
                    det_boxes.append((x1 + x, y1 + y, x2 + x, y2 + y))
                    det_scores.append(float(b.conf[0])); det_classes.append(int(b.cls[0]))
        keep = nms_boxes_xyxy(det_boxes, det_scores, conf_thr=conf, iou_thr=iou)
        for i in range(len(det_boxes)):
            if i not in keep: continue
            x1,y1,x2,y2 = map(int, det_boxes[i]); score = det_scores[i]
            cv2.rectangle(frame, (x1,y1), (x2,y2), (0,255,0), 2)
            cv2.putText(frame, f"drone {score:.2f}", (x1, max(0,y1-5)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
        out.write(frame); pbar.update(1)
    pbar.close(); cap.release(); out.release(); print("Gespeichert:", dst_video)

best_weights = find_best_weights() if 'find_best_weights' in globals() else None
videos = sorted(glob.glob(os.path.join(RAW_VIDEOS, "*")))
if best_weights and videos:
    src = videos[0]
    dst = f"/content/runs/detect/tiled_{os.path.splitext(os.path.basename(src))[0]}.mp4"
    tiled_inference_video(best_weights, src, dst, tile=640, overlap=0.2, conf=0.25, iou=0.5)
else:
    print("⚠️ Tiled Inference übersprungen (kein best.pt oder kein Video).")

## 1️⃣5️⃣ Ergebnisse nach Google Drive sichern
Kopiert `best.pt`, Vorhersagen und ggf. `tiled_*.mp4` nach Drive.

In [ ]:
os.makedirs(DRIVE_EXPORT_DIR, exist_ok=True)

def copy_if_exists(src_path, dst_dir):
    if src_path and os.path.exists(src_path):
        dst = os.path.join(dst_dir, os.path.basename(src_path))
        shutil.copy2(src_path, dst); print("kopiert →", dst)

best_weights = find_best_weights() if 'find_best_weights' in globals() else None
copy_if_exists(best_weights, DRIVE_EXPORT_DIR)

pred_dirs = sorted(glob.glob("/content/runs/detect/predict*"), key=os.path.getmtime)
for pd in pred_dirs[-2:]:
    dst = os.path.join(DRIVE_EXPORT_DIR, os.path.basename(pd))
    if os.path.exists(dst): shutil.rmtree(dst)
    shutil.copytree(pd, dst); print("kopiert Ordner →", dst)

for tv in glob.glob("/content/runs/detect/tiled_*.mp4"):
    copy_if_exists(tv, DRIVE_EXPORT_DIR)

print("✅ Export abgeschlossen. Drive Ordner:", DRIVE_EXPORT_DIR)

## 1️⃣6️⃣ (Optional) Umgebung protokollieren

In [ ]:
import sys
print("Python:", sys.version)
print("Letzte 50 Pakete:")
!pip freeze | tail -n 50